# Fast Filtering with Large Option Panels: Implications for Asset Pricing

As we don't have access to the data used in the paper, we will simulate data from the option pricing models whown in the figures (page 59 environ) and then estimate the model given the artificial data.

In [ ]:
import matplotlib.pyplot as plt

import simulation

## 1. Simulating data

We will first simulate the data from the simple Stochastic Volatility model of Heston. The model is given by the following SDEs:

$$
\begin{aligned}
dS_t &= \mu S_t dt + \sqrt{V_t} S_t dW_t^S \\
dV_t &= \kappa (\theta - V_t) dt + \sigma \sqrt{V_t} dW_t^v
\end{aligned}
$$

where $W_t^S$ and $W_t^v$ are two correlated Brownian motions with correlation $\rho$. We will simulate the data for the following parameters:

In [ ]:
# Simulate
time, S, V = simulate_heston(S0, V0, mu, kappa, theta, sigma, rho, T, N, dt)

# Plot results
plt.figure(figsize=(12, 6))
plt.subplot(2, 1, 1)
plt.plot(time, S, label="Stock Price")
plt.title("Heston Model Simulation")
plt.xlabel("Time")
plt.ylabel("Stock Price")
plt.grid()
plt.legend()

plt.subplot(2, 1, 2)
plt.plot(time, V, label="Variance", color="orange")
plt.xlabel("Time")
plt.ylabel("Variance")
plt.grid()
plt.legend()

plt.tight_layout()
plt.show()